# What is about ?


Claim: For higher dimensions hubs tends to be centered near cluster centers. 


--------------------------------------------------------------------------------------


Content of different versions of the notebook:

V8 - same as V6,7 for dimension 100

V7 - same as V6 for dimension 20

V6 - compare Gauss and uniform on the same plot. Dimension only 10. 

V4,5 - Guassian datacloud. V4 and V5 repeat same simulation to estimate variability on random effects.

V1,3 - uniform datacloud. Both versions - same similation to estimate variability on random effects.

Notebooks 1-5 simulate with params:

n_sample = 10** 5

list_dims = [2,5,8,10,20]

list_n_neighbors = [10]



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import minimum_spanning_tree

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time 


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
n_sample = 10**5
list_distribution_type = ['Gauss' , 'uniform' ]# 
#distribution_type =  'Gauss' #  'uniform' #      'Gauss' #
list_dims = [100]# ,5,8, 10,20,50]#list(range(2,200))+list(range(200,1000,100))
list_n_neighbors = [10]

verbose = 1 #  1000

df_stat = pd.DataFrame()
df_stat_short = pd.DataFrame()
df_stat_short4rk =  pd.DataFrame()
#list_save_results = []
#n_neighbors = 3

t00 = time.time()
t0 = time.time()


bins = range(0,15)
for distribution_type in list_distribution_type:
    for n_neighbors in list_n_neighbors:
      for d in list_dims :#[2,10]:
        for trial in range(1):
            if distribution_type == 'Gauss':
                X = np.random.randn(n_sample,d)
                if verbose >= 100:
                    print('Gauss')
            else:
                X = np.random.uniform(-1,1, size = (n_sample,d) )
                if verbose >= 100:
                    print('uniform')
            if verbose >= 100:
                print(X.shape,'X.shape')
            df_tmp = pd.DataFrame(X)
            df_tmp['Norm'] = np.sum(X**2, axis = 1) 
            df_tmp['rank'] = df_tmp['Norm'].rank()
            vec_rk = df_tmp['rank'].values

            t0 = time.time()
            m = kneighbors_graph(X, n_neighbors= n_neighbors, mode='connectivity' , include_self=False) #'distance' mode=  'connectivity'
            if verbose >= 100:
                print(time.time() - t0, 'secs passed on kNN calc')

            vec_degs = np.sum(m,axis = 0 )#.reshape(-1,1) #  get_degree_vector_from_adj_matrix(m)
            vec_degs = np.squeeze(np.array(vec_degs.ravel()))
            if verbose >= 100:
                print(time.time() - t0, 'secs passed, Degree Calc Finished')

            for thres in range(500):# [0,1,2,3,4,5]:
              mask = vec_degs >=   thres # * n_neighbors 
              if mask.sum() < 100:
                    break
              #print( vec_degs[mask] )
              mn = np.mean( np.sqrt( np.sum(X[mask,:]**2, axis = 1 )/ d ) )
              if 0:
                print(np.sum(mask) , thres, 'np.sum(mask) , thres,' )
                print(mn,'mn')

              # +' '+ deg_type  
              s = 'D'+str(d)+' n_neighbors'+str(n_neighbors)  +' '+distribution_type + ' n_sample' +str(n_sample) 
                #  +' trial'+str(trial)
              df_stat.loc[s, thres ] = mn
              df_stat_short.loc[s, thres ] = mn
              df_stat_short4rk.loc[s, thres ] = np.mean( vec_rk[mask] )
              df_stat.loc[s, str(thres)+' MeanRank' ] = np.mean( vec_rk[mask] )
              df_stat.loc[s, str(thres)+' SetSize' ] = np.sum(mask)
              df_stat.loc[s, str(thres)+' SetSizeNormed' ] = np.sum(mask) / len(X)
        if verbose >= 1:
          print('Dim',d,'finished.', np.round(time.time() - t00,1) , 'total seconds passed' )
    
  
    
plt.style.use('ggplot')
    
plt.figure(figsize = (15, 6 ))
for i in df_stat_short.index:
  v = df_stat_short.loc[i,:]
  plt.plot(v, '*-', label = str(i))
#s = 'D'+str(d)+' n_neighbors'+str(n_neighbors)+' '+ deg_type  +' '+distribution_type + ' n_sample' +str(n_sample)
#plt.title(s)
plt.ylabel('Average ditance to origin')
plt.xlabel('Degree Above that value')
plt.legend()
plt.show()

plt.figure(figsize = (15, 6 ))
for i in df_stat_short4rk.index:
  v = df_stat_short4rk.loc[i,:]
  plt.plot(v,'*-', label = str(i))
#s = 'D'+str(d)+' n_neighbors'+str(n_neighbors)+' '+ deg_type  +' '+distribution_type + ' n_sample' +str(n_sample)
#plt.title(s)
plt.ylabel('Average rank to origin')
plt.xlabel('Degree Above that value')
plt.title('Average Rank plot ')
plt.legend()
plt.show()

plt.figure(figsize = (15, 6 ))
for i in df_stat_short.index:
  v = df_stat_short.loc[i,:] 
  plt.loglog(v, '*-', label = str(i))
#s = 'D'+str(d)+' n_neighbors'+str(n_neighbors)+' '+ deg_type  +' '+distribution_type + ' n_sample' +str(n_sample)
plt.title('LogLog Plot')
plt.ylabel('Average ditance to origin')
plt.xlabel('Degree Above that value')
plt.legend()
plt.show()

print(time.time() - t00, 'total seconds passed' )

df_stat    

In [ ]:
df_stat.to_csv('df_stat.csv')
df_stat_short.to_csv('df_stat_short.csv')
df_stat_short4rk.to_csv('df_stat_short4rk.csv')


In [ ]:
fig = plt.figure(figsize = (18, 8 ))
fig.add_subplot(1, 2, 1 )
for i in df_stat_short.index:
  v = df_stat_short.loc[i,:]
  plt.plot(v,'*-', label = str(i).replace(' trial0',''))
#s = 'D'+str(d)+' n_neighbors'+str(n_neighbors)+' '+ deg_type  +' '+distribution_type + ' n_sample' +str(n_sample)
#plt.title(s)
plt.ylabel('Average ditance to origin')
plt.xlabel('Degree above that value')
#plt.title('Average Distance to origin plot ')
plt.title('A                                                    ',fontsize=30)
plt.legend()
#plt.show()

#plt.figure(figsize = (15, 6 ))
fig.add_subplot(1, 2, 2 )
for i in df_stat_short4rk.index:
  v = df_stat_short4rk.loc[i,:]
  plt.plot(v,'*-', label = str(i).replace(' trial0',''))
#s = 'D'+str(d)+' n_neighbors'+str(n_neighbors)+' '+ deg_type  +' '+distribution_type + ' n_sample' +str(n_sample)
#plt.title(s)
plt.ylabel('Average rank to origin')
plt.xlabel('Degree above that value')
plt.title('B                                                    ',fontsize=30)
plt.legend()
plt.show()



In [ ]:
fig.savefig('hubs_positions')